<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/%EC%8B%9C%EA%B3%84%EC%97%B4_1%ED%8C%80_0612_private_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test 데이터 전처리 후 inference
=> ulsan_weather_06, dangjin_weather_06 최신 업데이트 버전으로 가져오기

* 이미지 한 장이 하루여야 함
* 72시간의 데이터를 갖고 73번째 시간대의 데이터를 예측하는것

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at ./gdrive


In [ ]:
'''
dangjin_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_dangjin_fcst.csv")
ulsan_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_ulsan_fcst.csv")
energy = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_energy.csv")
'''

In [ ]:
dangjin_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/dangjin_weather_06.csv", encoding = 'utf-8')
ulsan_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/ulsan_weather_06.csv")

In [ ]:
dangjin_weather_06[-73:]

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed,방위각,고도,적경,적위,WindDirection_x,WindDirection_y,Wind_x,Wind_y,hour,day,month
142,2021-06-13 23:00:00,95.0,8.000000,19.000000,0.0,0.400000,335 57 54.52,-25 45 01.53,05 29 32.56,23 15 00.58,6.123234e-17,1.000000,2.449294e-17,0.400000,23,13,6
143,2021-06-14 00:00:00,97.0,6.000000,18.900000,160.0,0.500000,351 00 57.71,-29 09 44.12,05 29 43.07,23 15 07.97,3.420201e-01,-0.939693,1.710101e-01,-0.469846,0,14,6
144,2021-06-14 01:00:00,98.0,8.000000,18.500000,250.0,1.500000,006 48 55.41,-29 23 15.50,05 29 53.59,23 15 15.45,-9.396926e-01,-0.342020,-1.409539e+00,-0.513030,1,14,6
145,2021-06-14 02:00:00,98.0,8.000000,18.100000,0.0,0.400000,022 02 16.78,-26 23 31.71,05 30 04.10,23 15 23.04,6.123234e-17,1.000000,2.449294e-17,0.400000,2,14,6
146,2021-06-14 03:00:00,98.0,6.000000,18.400000,0.0,0.300000,035 40 35.51,-20 35 55.55,05 30 14.60,23 15 30.70,6.123234e-17,1.000000,1.836970e-17,0.300000,3,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2021-06-16 19:00:00,60.0,2.333333,25.666667,85.0,3.866667,292 28 13.62,09 06 40.45,05 41 18.89,23 21 57.08,9.961947e-01,0.087156,3.851953e+00,0.337002,19,16,6
211,2021-06-16 20:00:00,65.0,1.666667,24.333333,83.0,3.533333,301 11 01.29,-1 33 58.14,05 41 29.32,23 22 01.18,9.925462e-01,0.121869,3.506996e+00,0.430605,20,16,6
212,2021-06-16 21:00:00,70.0,1.000000,23.000000,81.0,3.200000,311 02 50.96,-11 14 13.02,05 41 39.79,23 22 05.28,9.876883e-01,0.156434,3.160603e+00,0.500590,21,16,6
213,2021-06-16 22:00:00,70.0,1.000000,23.000000,81.0,3.200000,322 30 59.74,-19 26 29.04,05 41 50.27,23 22 09.43,9.876883e-01,0.156434,3.160603e+00,0.500590,22,16,6


In [ ]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
  #dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
  #dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  #dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
  #dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
  return dangjin_fcst

In [ ]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [ ]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))


In [ ]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  #Q. 어차피 다 나눠놨는데 싸인 코싸인은 왜 씌우는것..?
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']

dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

#dangjin_weather_06['WindDirection_catg'] = dangjin_weather_06['WindDirection'].apply(angle_to_dir)
#ulsan_weather_06['WindDirection_catg'] = ulsan_weather_06['WindDirection'].apply(angle_to_dir)

In [ ]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [ ]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [ ]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [ ]:
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [ ]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각',	'고도',	'적경',	'적위']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [ ]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,19.100000,15.952873,82.0,9.000000,1.469576e-16,-2.400000,1,6,7,-29.4636,5,22.5000
1,18.600000,16.391992,87.0,9.000000,1.710101e-01,-0.469846,2,6,22,-26.4148,5,22.5014
2,18.100000,16.606323,91.0,9.000000,2.449294e-17,0.400000,3,6,36,-20.4900,5,22.5027
3,18.700000,17.024790,90.0,9.000000,2.052121e-01,-0.563816,4,6,47,-12.4718,5,22.5041
4,17.900000,17.086746,95.0,8.000000,0.000000e+00,0.000000,5,6,58,-3.1341,5,22.5055
...,...,...,...,...,...,...,...,...,...,...,...,...
210,25.666667,17.319098,60.0,2.333333,3.851953e+00,0.337002,19,6,292,9.0640,5,23.2157
211,24.333333,17.328438,65.0,1.666667,3.506996e+00,0.430605,20,6,301,-1.3358,5,23.2201
212,23.000000,17.231231,70.0,1.000000,3.160603e+00,0.500590,21,6,311,-11.1413,5,23.2205
213,23.000000,17.231231,70.0,1.000000,3.160603e+00,0.500590,22,6,322,-19.2629,5,23.2209


In [ ]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71)     

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,...,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.100000,15.952873,82.0,9.000000,1.469576e-16,-2.400000,1,6,7,-29.4636,5,22.5000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.100000,15.952873,82.0,9.000000,1.469576e-16,-2.400000,1.0,6.0,7.0,-29.4636,5.0,22.5000,18.600000,16.391992,87.0,9.000000,1.710101e-01,-0.469846,2,6,22,-26.4148,5,22.5014
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19.100000,15.952873,82.000000,9.000000,1.469576e-16,-2.400000,1.0,6.0,7.0,-29.4636,5.0,22.5000,18.600000,16.391992,87.0,9.000000,1.710101e-01,-0.469846,2.0,6.0,22.0,-26.4148,5.0,22.5014,18.100000,16.606323,91.0,9.000000,2.449294e-17,0.400000,3,6,36,-20.4900,5,22.5027
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,-29.4636,5.0,22.5000,18.600000,16.391992,87.000000,9.000000,1.710101e-01,-0.469846,2.0,6.0,22.0,-26.4148,5.0,22.5014,18.100000,16.606323,91.0,9.000000,2.449294e-17,0.400000,3.0,6.0,36.0,-20.4900,5.0,22.5027,18.700000,17.024790,90.0,9.000000,2.052121e-01,-0.563816,4,6,47,-12.4718,5,22.5041
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.0,-26.4148,5.0,22.5014,18.100000,16.606323,91.000000,9.000000,2.449294e-17,0.400000,3.0,6.0,36.0,-20.4900,5.0,22.5027,18.700000,17.024790,90.0,9.000000,2.052121e-01,-0.563816,4.0,6.0,47.0,-12.4718,5.0,22.5041,17.900000,17.086746,95.0,8.000000,0.000000e+00,0.000000,5,6,58,-3.1341,5,22.5055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,19.2,16.981900,87.0,7.0,4.788282e-01,1.315570,20.0,6.0,301.0,-1.4528,5.0,23.1438,18.3,17.316595,94.0,7.0,7.660444e-01,0.642788,21.0,6.0,311.0,-11.2532,5.0,23.1446,18.4,17.583632,95.0,8.0,-1.285879e-16,0.700000,22.0,6.0,322.0,-19.3719,5.0,23.1453,19.0,18.179888,95.0,8.0,...,267.0,44.1012,5.0,23.2144,27.666667,17.317977,53.333333,2.333333,4.324463e+00,0.277124,17.0,6.0,276.0,32.1314,5.0,23.2148,27.000000,17.187066,55.0,3.000000,4.194244e+00,0.219811,18.0,6.0,284.0,20.2731,5.0,23.2152,25.666667,17.319098,60.0,2.333333,3.851953e+00,0.337002,19,6,292,9.0640,5,23.2157
211,18.3,17.316595,94.0,7.0,7.660444e-01,0.642788,21.0,6.0,311.0,-11.2532,5.0,23.1446,18.4,17.583632,95.0,8.0,-1.285879e-16,0.700000,22.0,6.0,322.0,-19.3719,5.0,23.1453,19.0,18.179888,95.0,8.0,2.449294e-17,0.400000,23.0,6.0,335.0,-25.4501,5.0,23.1500,18.9,18.412749,97.0,6.0,...,276.0,32.1314,5.0,23.2148,27.000000,17.187066,55.000000,3.000000,4.194244e+00,0.219811,18.0,6.0,284.0,20.2731,5.0,23.2152,25.666667,17.319098,60.0,2.333333,3.851953e+00,0.337002,19.0,6.0,292

In [ ]:
#앞에 72개는 nan 제거

dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,...,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
71,19.1,15.952873,82.0,9.0,1.469576e-16,-2.400000,1.0,6.0,7.0,-29.4636,5.0,22.5000,18.6,16.391992,87.0,9.0,1.710101e-01,-0.469846,2.0,6.0,22.0,-26.4148,5.0,22.5014,18.1,16.606323,91.0,9.0,2.449294e-17,0.400000,3.0,6.0,36.0,-20.4900,5.0,22.5027,18.7,17.024790,90.0,9.0,...,311.0,-11.3926,5.0,23.0345,20.800000,18.737684,88.000000,10.000000,1.162868e+00,-3.194955,22.0,6.0,322.0,-19.5108,5.0,23.0355,20.400000,18.161646,87.0,10.000000,3.735173e-16,-6.100000,23.0,6.0,336.0,-25.5814,5.0,23.0406,20.700000,19.176528,91.0,10.000000,3.673940e-16,-6.000000,0,6,351,-29.2138,5,23.0416
72,18.6,16.391992,87.0,9.0,1.710101e-01,-0.469846,2.0,6.0,22.0,-26.4148,5.0,22.5014,18.1,16.606323,91.0,9.0,2.449294e-17,0.400000,3.0,6.0,36.0,-20.4900,5.0,22.5027,18.7,17.024790,90.0,9.0,2.052121e-01,-0.563816,4.0,6.0,47.0,-12.4718,5.0,22.5041,17.9,17.086746,95.0,8.0,...,322.0,-19.5108,5.0,23.0355,20.400000,18.161646,87.000000,10.000000,3.735173e-16,-6.100000,23.0,6.0,336.0,-25.5814,5.0,23.0406,20.700000,19.176528,91.0,10.000000,3.673940e-16,-6.000000,0.0,6.0,351.0,-29.2138,5.0,23.0416,20.700000,19.698490,94.0,10.000000,2.632991e-16,-4.300000,1,6,6,-29.3306,5,23.0427
73,18.1,16.606323,91.0,9.0,2.449294e-17,0.400000,3.0,6.0,36.0,-20.4900,5.0,22.5027,18.7,17.024790,90.0,9.0,2.052121e-01,-0.563816,4.0,6.0,47.0,-12.4718,5.0,22.5041,17.9,17.086746,95.0,8.0,0.000000e+00,0.000000,5.0,6.0,58.0,-3.1341,5.0,22.5055,17.6,17.279817,98.0,6.0,...,336.0,-25.5814,5.0,23.0406,20.700000,19.176528,91.000000,10.000000,3.673940e-16,-6.000000,0.0,6.0,351.0,-29.2138,5.0,23.0416,20.700000,19.698490,94.0,10.000000,2.632991e-16,-4.300000,1.0,6.0,6.0,-29.3306,5.0,23.0427,20.600000,20.272411,98.0,10.000000,-4.289849e+00,-3.599611,2,6,22,-26.3053,5,23.0437
74,18.7,17.024790,90.0,9.0,2.052121e-01,-0.563816,4.0,6.0,47.0,-12.4718,5.0,22.5041,17.9,17.086746,95.0,8.0,0.000000e+00,0.000000,5.0,6.0,58.0,-3.1341,5.0,22.5055,17.6,17.279817,98.0,6.0,4.698463e-01,0.171010,6.0,6.0,66.0,7.2256,5.0,22.5109,19.4,17.178530,87.0,2.0,...,351.0,-29.2138,5.0,23.0416,20.700000,19.698490,94.000000,10.000000,2.632991e-16,-4.300000,1.0,6.0,6.0,-29.3306,5.0,23.0427,20.600000,20.272411,98.0,10.000000,-4.289849e+00,-3.599611,2.0,6.0,22.0,-26.3053,5.0,23.0437,19.800000,19.474394,98.0,10.000000,-2.017919e+00,-5.544186,3,6,35,-20.4047,5,23.0448
75,17.9,17.086746,95.0,8.0,0.000000e+00,0.000000,5.0,6.0,58.0,-3.1341,5.0,22.5055,17.6,17.279817,98.0,6.0,4.698463e-01,0.171010,6.0,6.0,66.0,7.2256,5.0,22.5109,19.4,17.178530,87.0,2.0,8.457234e-01,0.307818,7.0,6.0,75.0,18.4130,5.0,22.5123,21.9,18.291218,80.0,1.0,...,6.0,-29.3306,5.0,23.0427,20.600000,20.272411,98.000000,10.000000,-4.289849e+00,-3.599611,2.0,6.0,22.0,-26.3053,5.0,23.0437,19.800000,19.474394,98.0,10.000000,-2.017919e+00,-5.544186,3.0,6.0,35.0,-20.4047,5.0,23.0448,19.600000,19.274889,98.0,9.000000,-1.231273e+00,-3.382893,4,6,47,-12.4133,5,23.0459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,19.2,16.981900,87.0,7.0,4.788282e-01,1.315570,20.0,6.0,301.0,-1.4528,5.0,23.1438,18.3,17.316595,94.0,7.0,7.660444e-01,0.642788,21.0,6.0,311.0,-11.2532,5.0,23.1446,18.4,17.583632,95.0,8.0,-1.285879e-16,0.700000,22.0,6.0,322.0,-19.3719,5.0,23.1453,19.0,18.179888,95.0,8.0,...,267.0,44.1012,5.0,23.2144,27.666667

In [ ]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [ ]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [ ]:
import tensorflow as tf

In [ ]:
dangjin_weather_06.shape

(144, 864)

In [ ]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 12, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 12, 1))

In [ ]:
dangjin_weather_06.shape

(144, 72, 12, 1)

In [ ]:
'''
dangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) 
dangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))

ulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))
ulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))
'''

'\ndangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) \ndangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))\n\nulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))\nulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))\n'

In [ ]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [ ]:
ulsan_weather_06.shape

(144, 72, 12, 3)

In [ ]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

In [ ]:
dangjin_test.shape

(24, 72, 12, 3)

* Inference

In [ ]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [ ]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [ ]:
floating3

array([[ 31.853054],
       [ 30.95657 ],
       [ 31.660776],
       [ 35.733974],
       [ 32.376926],
       [ 24.873577],
       [ 29.571812],
       [ 47.035183],
       [ 70.9418  ],
       [ 99.40091 ],
       [125.37739 ],
       [150.43596 ],
       [172.52393 ],
       [175.77605 ],
       [163.15366 ],
       [155.1758  ],
       [130.914   ],
       [ 96.119736],
       [ 61.25092 ],
       [ 45.982685],
       [ 33.779247],
       [ 32.754436],
       [ 40.320633],
       [ 66.35559 ]], dtype=float32)

In [ ]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [ ]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [ ]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [ ]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [ ]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [ ]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [ ]:
floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5

In [ ]:
sample_submission2 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/neural_prophet_validation.csv")

In [ ]:
sample_submission2

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0,0,0,0
1388,2021-07-08 21:00:00,0,0,0,0
1389,2021-07-08 22:00:00,0,0,0,0
1390,2021-07-08 23:00:00,0,0,0,0


In [ ]:
sample_submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.000000,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.000000,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.000000,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.000000,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.000000,0.0,0.651203
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158


* 시간은 내일 00시부터 23시까지 !! (가져온 날씨 데이터가 그럼)

In [ ]:
#sample_submission.loc[743:766] #6/12일
#sample_submission.loc[767:790] #6/13일    #loc는 끝 인덱스도 포함....
#sample_submission.loc[815:839] #6/14일
sample_submission.loc[839:862] #6/15일

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
839,2021-06-15 24:00:00,27.941643,32.362556,44.454659,24.855354
840,2021-06-16 01:00:00,27.255981,33.765167,47.097160,25.229973
841,2021-06-16 02:00:00,24.583191,33.574669,45.903938,24.723541
842,2021-06-16 03:00:00,25.427437,34.455673,46.157692,23.635098
843,2021-06-16 04:00:00,27.851814,36.121338,51.425285,24.337521
844,2021-06-16 05:00:00,26.676918,35.106033,54.524750,25.226013
845,2021-06-16 06:00:00,23.900604,32.598877,50.802551,23.452633
846,2021-06-16 07:00:00,29.678085,38.144707,54.211716,26.576847
847,2021-06-16 08:00:00,58.886688,65.204926,91.256210,50.977913
848,2021-06-16 09:00:00,126.523972,122.388519,177.697647,101.595581


In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[839:862].shape

(24, 5)

In [ ]:
sample_submission.loc[839:862, 'dangjin_floating'] = floating
sample_submission.loc[839:862, 'dangjin_warehouse'] = warehouse
sample_submission.loc[839:862, 'dangjin'] = dangjin
sample_submission.loc[839:862, 'ulsan'] = ulsan

In [ ]:
sample_submission.loc[839:862]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
839,2021-06-15 24:00:00,13.309427,10.056867,22.010944,17.552078
840,2021-06-16 01:00:00,14.529297,13.778664,24.223368,15.918968
841,2021-06-16 02:00:00,14.827273,17.487392,25.515011,15.798373
842,2021-06-16 03:00:00,15.341497,25.989136,29.383408,16.686413
843,2021-06-16 04:00:00,14.278305,34.209038,30.492401,18.072704
844,2021-06-16 05:00:00,13.100940,40.996075,33.683434,20.001499
845,2021-06-16 06:00:00,15.867317,45.348000,37.714039,24.560970
846,2021-06-16 07:00:00,25.228296,50.492554,56.243645,36.399311
847,2021-06-16 08:00:00,45.281792,59.841084,92.524734,67.512878
848,2021-06-16 09:00:00,68.298080,69.563965,138.692535,121.560410


In [ ]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0615_submission.csv", index = False)

In [ ]:
ls

0608_cnn_5fold.csv  gdrive/  sample_data/
